# 필요한 일
- 특정 좌표를 반경으로 해당 반경하는 점 1,2,3 ..들을 찾고
- => 해당 점 1,2,3 들의 값을 파악(예컨대, 해당 점 위치의 상주인구)
- => 특정 좌표 반경의 상주인구 수를 합산, 또한 집객시설도 추후 합산(이건 아마 단순 count가 될듯)
- 특정 좌표를 임의로 찍을지 vs 특정 좌표를 특정 구의 특정 행정동 전체를 격자모양으로 나눠서 전수조사
    - 지역구의 대상 행정동을 정하는 방법은 폐플라스틱, 고철 캔의 수가 많은 동 선택 예정(N개 미정)

## 예시 데이터 프레임 만들기 그냥 강남구청 근처 좌표 상하좌우 1km로 예시

In [1]:
import random
import pandas as pd

# 랜덤 시드 설정
random.seed(28)

# 데이터 생성
building_types = ["아파트", "빌라", "빌라", "빌라", "빌라", "주택"]
residents = {"아파트": (100, 200), "빌라": (16, 25), "주택": (2, 5)}

data = []

for _ in range(100):
    building_type = random.choice(building_types)
    min_residents, max_residents = residents[building_type]
    num_residents = random.randint(min_residents, max_residents)
    x_coordinate = round(random.uniform(127.0361778, 127.0591223), 8)
    y_coordinate = round(random.uniform(37.5066263, 37.5263467), 8)
        
    data.append([building_type, num_residents, x_coordinate, y_coordinate])

# 데이터프레임 생성
df = pd.DataFrame(data, columns=["건물 구분", "거주 인구 수", "x 좌표", "y 좌표"])

# 결과 출력
df

,건물 구분,거주 인구 수,x 좌표,y 좌표
0,아파트,195,127.039180,37.518405
1,빌라,19,127.039218,37.515779
2,빌라,19,127.041073,37.514350
3,빌라,25,127.040482,37.507174
4,빌라,17,127.038357,37.526046
...,...,...,...,...
95,빌라,21,127.056787,37.522695
96,주택,2,127.055372,37.523314
97,주택,3,127.050486,37.512246
98,아파트,164,127.053908,37.519472


In [2]:
df['건물 구분'].value_counts()

건물 구분
빌라     67
주택     21
아파트    12
Name: count, dtype: int64

## 한글..설치 ㅎ..

In [3]:
from matplotlib import rc, font_manager as fm

# 나눔고딕 폰트 경로 설정
font_path = '../나눔고딕/NanumFontSetup_TTF_GOTHIC/NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path).get_name()

# 폰트 설정
rc('font', family=font_name)

## 폴리움 시작

In [8]:
import folium
import pandas as pd

# 지도 중심 위치 설정
map_center = [37.5173050, 127.0475020]

# Folium 지도 생성
m = folium.Map(location=map_center, zoom_start=13)

# 데이터프레임의 좌표를 마커로 추가
for index, row in df.iterrows():
    folium.Marker([row['y 좌표'], row['x 좌표']]).add_to(m)

# 지도를 HTML 파일로 저장하거나, 노트북에 표시
m.save('map.html')  # HTML 파일로 저장

# 노트북에서 표시
m


In [17]:
import folium
from folium.plugins import MarkerCluster
from geopy.distance import great_circle

# 시작 경도, 위도 및 종료 경도, 위도 설정
start_lon, end_lon = 127.0361778, 127.0591223
start_lat, end_lat = 37.5066263, 37.5263467

# 100미터의 거리를 표현하기 위해 0.0009
distance = 0.0009

# 지도 생성
m2 = folium.Map(location=[start_lat, start_lon], zoom_start=14)

# MarkerCluster 객체 생성
marker_cluster = MarkerCluster().add_to(m2)

# 점 찍기
for lat in range(int(start_lat * 1000000), int(end_lat * 1000000), int(distance * 1000000)):
    for lon in range(int(start_lon * 1000000), int(end_lon * 1000000), int(distance * 1000000)):
        lat = lat / 1000000
        lon = lon / 1000000
        folium.Marker([lat, lon]).add_to(marker_cluster)

# 지도를 HTML 파일로 저장
m2.save('map2.html')
m2